# Import Libraries

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display_html

# Google sheet 

## Authorization

In [2]:
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

## Function

In [3]:
my_spreadsheet_url='https://docs.google.com/spreadsheets/d/1GhUEANcfCKYnrjCQbWEddX_U56Kh_GrQHrCK0NurlUo/edit#gid=0'
my_workbook = 'Sheet1'
wks = gc.open_by_url(my_spreadsheet_url)

def get_input_file(spreadsheet_url, worksheet_name):
    sources_spreadsheet = gc.open_by_url(spreadsheet_url)
    sources_worksheet = sources_spreadsheet.worksheet(worksheet_name)
    input_file = pd.DataFrame(sources_worksheet.get_all_records())
    return input_file

# Load Data

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
df = pd.read_csv('Data_Quality.csv')

Saving Data_Quality.csv to Data_Quality (3).csv
User uploaded file "Data_Quality.csv" with length 519 bytes


In [5]:
df

,Unnamed: 0,Category,Rank,Unique_Identifier,Seller_State,Seller_City,Postal_Code
0,0,Phone,14,X12JI92X,Karnataka,Bangalore,560004
1,1,Tab,53,ABXI98IB,Kerala,Kannur,670001
2,2,Phone,55,PPQ129IN,Karnataka,Magadi,560023
3,3,Phone,3.9,KLIO9900,Karnataka,Mangalore,575003
4,4,Phone,57,SBX9INJ2,Kerala,Trivandrum,695002
5,5,Phone,8,UIXCV12I,Karnataka,Trivandrum,695003
6,6,Phone,5,12ICVBNX,Kerala,Bangalore,560005
7,7,Phone,VC,ABC$5678,Karnataka,Bangalore,560009
8,8,Phone,51,LFFVB123,Karnataka,Kannur,560001
9,9,Phone,7,BBXR66G,Karnataka,Bangalore,67890


# Data Quality Checks for Accuracy

## Category

In [6]:
display_html(f"<h3><br/>Accuray Checks for Cateogory", raw=True)
Expected_Category = 'Phone'+ '|'+ 'Tab' +'|' + 'Laptop' 
print("Expected_Category are : ", Expected_Category)
if(sum(~df['Category'].str.contains(Expected_Category, na =False))==0):
  display_html(f"<h3><br/>✅  - No category apart from Phone, Tab or Laptop", raw=True)
else:
  display_html(f"<h3><br/>❌Below records does belong to expected category", raw=True)
  display(df[~df['Category'].str.contains(Expected_Category, na =False)][['Category']])

Accuray Checks for Cateogory

Expected_Category are :  Phone|Tab|Laptop


✅ - No category apart from Phone, Tab or Laptop

## Rank

In [7]:
display_html(f"<h3><br/>Accuracy Checks for Rank", raw=True)
# Check 1
if(sum(df['Rank'].apply(str).str.isdigit() == False)==0):
  display_html(f"<h3><br/>Check 1 : ✅  Rank have only Integer Values", raw=True)
else:
  display_html(f"<h3><br/>Check 1 : ❌ Ranks which does not have Integer Values", raw=True)
  display(df[df['Rank'].apply(str).str.isdigit() == False][['Category','Rank']])

# Check 2 -- 
temp_df = df[df['Rank'].apply(str).str.isdigit() == True]
if(temp_df.shape[0]==df.shape[0]):
  display_html(f"<h3><br/>Check 2 : ✅  Rank with Integer Values falls under boundary values", raw=True)
else:
  display_html(f"<h3><br/>Check 2 : ❌ Ranks with Integer Values does not falls under boundary values ", raw=True)
  display(temp_df[~temp_df['Rank'].astype(int).between(1, 50)][['Category','Rank']])


Accuracy Checks for Rank

Check 1 : ❌ Ranks which does not have Integer Values

,Category,Rank
3,Phone,3.9
7,Phone,VC


Check 2 : ❌ Ranks with Integer Values does not falls under boundary values

,Category,Rank
1,Tab,53
2,Phone,55
4,Phone,57
8,Phone,51


## Unique_Identifier

In [8]:
display_html(f"<h3><br/>Accuracy Checks for Unique_Identifier", raw=True)
# Check 1
if(sum(df['Unique_Identifier'].str.isalnum() == False)==0):
  display_html(f"<h3><br/>Check 1 : ✅ Unique_Identifier is alphanumeric", raw=True)
else:
  display_html(f"<h3><br/>Check 1 : ❌ Unique_Identifier is not alphanumeric", raw=True)
  display(df[df['Unique_Identifier'].str.isalnum() == False][['Category','Unique_Identifier']])

# Check 2
temp_df = df[df['Unique_Identifier'].str.isalnum() == True]
if(sum(temp_df['Unique_Identifier'].str.len() != 8)==0):
  display_html(f"<h3><br/>Check 2 : ✅ Unique_Identifier which are alphanumeric have only 8 characters", raw=True)
else:
  display_html(f"<h3><br/>Check 2 : ❌ Unique_Identifier which are alphanumeric with character length not equal to 8", raw=True)
  display(temp_df[temp_df['Unique_Identifier'].str.len()!= 8][['Category','Unique_Identifier']])

Accuracy Checks for Unique_Identifier

Check 1 : ❌ Unique_Identifier is not alphanumeric

,Category,Unique_Identifier
7,Phone,ABC$5678


Check 2 : ❌ Unique_Identifier which are alphanumeric with character length not equal to 8

,Category,Unique_Identifier
9,Phone,BBXR66G


## Seller_State

In [9]:
display_html(f"<h3><br/>Accuracy Checks for Seller_State", raw=True)
print("Expected_Category are : ", 'Karnataka or Kerala')
if(sum(~df['Seller_State'].str.contains('Karnataka|Kerala', na =False))==0):
  display_html(f"<h3><br/>✅  - Seller belongs to Karnataka or Kerala ", raw=True)
else:
  display_html(f"<h3><br/>❌- Seller apart from Karnataka or Kerala", raw=True)
  display(df[~df['Seller_State'].str.contains('Karnataka|Kerala', na =False)][['Seller_State','Category']])

Accuracy Checks for Seller_State

Expected_Category are :  Karnataka or Kerala


✅ - Seller belongs to Karnataka or Kerala

## Seller_City

In [10]:
city_info = get_input_file(my_spreadsheet_url,my_workbook) #Google Sheet Data

In [11]:
display_html(f"<h3><br/>Accuracy Check for Seller_State - Seller City", raw=True)
State_list = ['Karnataka','Kerala']
postal_start= [5,6] # For Postal Code Checks
counter = 0
postal = df.copy()
for i in range(len(State_list)):
  df_cities = df[df['Seller_State']==State_list[i]]['Seller_City'].unique() # DataFrame Cities
  correct_cities = city_info[city_info['State']==State_list[i]]['Cities'].unique() # Cities From Google Sheet 
  wrng_city = [i for i in df_cities if i not in correct_cities]
  if(len(wrng_city)!=0):
    counter +=1
    display_html(f"<h3><br/>❌ - Check failed for {State_list[i]}", raw=True)
    print(f'\nWrong Mapping in cities list for {State_list[i]} is/are : {wrng_city}')
    display(df[(df['Seller_City'].isin(wrng_city)) & (df['Seller_State']==State_list[i])][['Seller_State','Seller_City']])
    postal = postal.drop(df[(df['Seller_City'].isin(wrng_city)) & (df['Seller_State']==State_list[i])].index)## Creating DataFrame for Postal Code Checks

if counter==0:
   display_html(f"<h3><br/>✅  All cities are mapped properly", raw=True) 


Accuracy Check for Seller_State - Seller City

❌ - Check failed for Karnataka


Wrong Mapping in cities list for Karnataka is/are : ['Trivandrum', 'Kannur']


,Seller_State,Seller_City
5,Karnataka,Trivandrum
8,Karnataka,Kannur


❌ - Check failed for Kerala


Wrong Mapping in cities list for Kerala is/are : ['Bangalore']


,Seller_State,Seller_City
6,Kerala,Bangalore


# Postal_Code

In [12]:
display_html(f"<h3><br/>Accuracy Check for Postal Code", raw=True)
#Check 1
if (len(postal[postal['Postal_Code'].apply(str).str.len()!= 6].index)!=0):
  display_html(f"<h3><br/>\nCheck 1- ❌ - Postal Code with less than 6 digits", raw=True)
  display(postal[postal['Postal_Code'].apply(str).str.len()!= 6])
  postal.drop(postal[postal['Postal_Code'].apply(str).str.len()!= 6].index, inplace=True)
else:
  display_html(f"<h3><br/>Check 1 - ✅  All Postal Code are 6 digits", raw=True) 

#Check 2
temp_df = pd.DataFrame()
for i in range(len(State_list)):
  temp_df =temp_df.append(postal[(postal['Seller_State']==State_list[i]) & (~postal['Postal_Code'].apply(str).str.startswith(\
                                                                                      str(postal_start[i])))])

if(temp_df.shape[0]==0): 
  display_html(f"<h3><br/>Check 2 - ✅  All Postal Code with 6 digits are properly mapped", raw=True)
else:
  display_html(f"<h3><br/>Check 2 -  ❌ Postal Code with 6 digits wrongly properly", raw=True)
  display(temp_df[['Seller_State','Postal_Code']])

Accuracy Check for Postal Code

Check 1- ❌ - Postal Code with less than 6 digits

,Unnamed: 0,Category,Rank,Unique_Identifier,Seller_State,Seller_City,Postal_Code
9,9,Phone,7,BBXR66G,Karnataka,Bangalore,67890


Check 2 - ✅ All Postal Code with 6 digits are properly mapped

In [13]:
df['Category'].value_counts()

Phone    9
Tab      1
Name: Category, dtype: int64

# Data Quality Check for Completeness

In [14]:
l1 = df['Category'].unique().tolist()
l1.sort()
l2 = Expected_Category.split('|')
if(l1==l2):
  display(f"<h3><br/> ✅  Data Set have all 3 categories", raw=True)
else:
  display_html(f"<h3><br/>❌ Data Set does not have all 3 categories", raw=True)
  display_html(f"<h3><br/>Expected categories are {Expected_Category.split('|')}, but we have only categories \
  {df['Category'].unique().tolist()}", raw=True)

❌ Data Set does not have all 3 categories

Expected categories are ['Phone', 'Tab', 'Laptop'], but we have only categories ['Phone', 'Tab']